In [1]:
import os


In [2]:
%pwd


'c:\\Users\\Achraf\\Desktop\\MASTER\\Python Avancé\\Mini-Projet\\mlops-fakenews-text-classification\\research'

In [3]:
os.chdir("../")


In [4]:
%pwd


'c:\\Users\\Achraf\\Desktop\\MASTER\\Python Avancé\\Mini-Projet\\mlops-fakenews-text-classification'